# NRPy+ Tutorial

**[Zachariah B. Etienne](http://math.wvu.edu/~zetienne/)** $\leftarrow$ Please feel free to email comments, revisions, or errata!

## Introduction & Motivation

NRPy+ ("Python-based code generation for numerical relativity... and beyond!") is a Python code that extends the C code output functionality of the [SymPy](http://www.sympy.org) computer algebra system, to minimize both human and computational effort when numerically solving [hyperbolic](https://en.wikipedia.org/wiki/Hyperbolic_partial_differential_equation) and [parabolic](https://en.wikipedia.org/wiki/Parabolic_partial_differential_equation) partial differential equations.

### Interactive NRPy+ Tutorials
#### (READ FIRST, IN ORDER) Basic functionality of NRPy+
+ #### [Basic C Code Output Options (including SIMD) / The Parameter Interface](Tutorial-Coutput__Parameter_Interface.ipynb)
+ #### [Indexed Expressions (e.g., tensors, pseudotensors, etc.)](Tutorial-Indexed_Expressions.ipynb)
+ #### [Numerical Grids](Tutorial-Numerical_Grids.ipynb)
+ #### [Finite Difference Derivatives](Tutorial-Finite_Difference_Derivatives.ipynb)
+ #### [Moving beyond Cartesian Grids: Reference Metric](Tutorial-Reference_Metric.ipynb)

#### (READ SECOND, IN ORDER) Worked Examples
+ #### [The Scalar Wave Equation (Cartesian Coordinates)](Tutorial-ScalarWave.ipynb)
+ #### [The Scalar Wave Equation in Curvilinear Coordinates](Tutorial-ScalarWave-Curvi.ipynb)
+ #### [General Relativity in the BSSN Formalism, Moving Puncture Gauge](Tutorial-BSSN.ipynb)

## Part 1: Generating C code to solve the scalar wave equation, in Cartesian coordinates

### Problem Statement

We wish to numerically solve the scalar wave equation in Cartesian coordinates:
$$\partial_t^2 u = c^2 \nabla^2 u \text{,}$$
where $u$ is a function of time and space: $u = u(t,x,y,...)$ (spatial dimension as-yet unspecified) and $c$ is the wave speed,

subject to some initial condition
$$u(0,x,y,...) = f(x,y,...)$$

and suitable, approximate spatial boundary conditions.

Define 
$$v(t,x,y,...) = \partial_t u(t,x,y,...)$$

In this way, the second-order PDE is reduced to a set of two coupled first-order PDEs

\begin{align}
\partial_t u &= v \\
\partial_t v &= c^2 \nabla^2 u.
\end{align}


### Numerical Solution to the Wave Equation 

We will find that breaking the second-order time derivative into two first-order, coupled equations in this way is quite convenient numerically, as it enables us to step forward in time using the Method of Lines. This method enables us to handle timestepping the same way we would handle an ODE integration, allowing us to use [Runge-Kutta methods](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) for timestepping. 

As for the $\nabla^2 u$ term, spatial derivatives are handled in NRPy+ via [finite differencing](https://en.wikipedia.org/wiki/Finite_difference).

We will sample the solution $\{u,v\}$ at discrete, uniformly-sampled points in space and time For simplicity, let's assume that we consider the wave equation in one spatial dimension. Then the solution at any sampled point in space and time is given by
$$u^n_i = u(t_n,x_i) = u(t_0 + n \Delta t, x_0 + i \Delta x),$$
where $\Delta t$ and $\Delta x$ represent the temporal and spatial resolution, respectively. $v^n_i$ is sampled at the same points in space and time.

#### Spatial Derivatives: the $\nabla^2$ Operator

To minimize complication, we will restrict ourselves to solving the wave equation in one spatial dimension, so
$$\nabla^2 u = \partial_x^2 u.$$
Extension of this operator to higher spatial dimensions is straightforward, particularly when using NRPy+.

We wish to approximate this derivative using [finite difference](https://en.wikipedia.org/wiki/Finite_difference). (FD) techniques. FD techniques are usually equivalent to first finding the unique $N$th degree polynomial that passes through $N+1$ sampled points of our function ($u$) in the neighborhood of where we wish to find the derivative. Then, provided $u$ is smooth and properly-sampled the derivative of the polynomial is an approximate derivative of $u$. The approximation error generally decreases as the polynomial degree or sampling density increases.

Finite difference derivatives are written in the form
$$\left[\partial_x u\right]_i = \sum_{j} \left(u_j\right) a_j,$$
where the $a_j$s are known as *finite difference coefficients*. For a given finite difference representation (corresponding to a fixed polynomial degree), the set of $a_j$s are unique.

For example, the second-order derivative $\partial_x^2$ accurate to fourth-order in uniform grid spacing $\Delta x$ (from fitting the unique 4th-degree polynomial to 5 sample points of $u$) is given by
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j = \frac{1}{(\Delta x)^2}
\left(
-\frac{1}{12} \left(u_{j+2} + u_{j-2}\right) 
+ \frac{4}{3}  \left(u_{j+1} + u_{j-1}\right)
- \frac{5}{2} u_j \right)
+ \mathcal{O}\left((\Delta x)^4\right).
\end{equation}

As another example, the same derivative accurate to second-order in uniform grid spacing $\Delta x$ (from fitting the unique 4th-degree polynomial to 3 sample points of $u$) is given by
\begin{equation}
\left[\partial_x^2 u(t,x)\right]_j = \frac{1}{(\Delta x)^2}
\left(u_{j+1} - 2 u_j + u_{j-1}\right)
+ \mathcal{O}\left((\Delta x)^2\right).
\end{equation}

For more details on how finite difference coefficients are computed (it's not magic!), as well as its implementation in NRPy+, see the Appendix at the bottom of this notebook.

#### Temporal Derivatives: Numerical Timestepping




In [1]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *

In [2]:
# The name of this module ("scalarwave") is given by __name__:
thismodule = __name__

par.set_parval_from_str("DIM",1)

# Step 0: Get the spatial dimension, as it was set in 
#         the grid module.
DIM = par.parval_from_str("DIM")

# Step 1: Register gridfunctions that are needed as input.
uu, vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 2: Declare the rank-2 indexed expression \partial_{ij} u,
#         which is symmetric about interchange of indices i and j
#         Derivative variables like these must have an underscore
#         in them, so the finite difference module can parse the
#         variable name properly.
uu_dDD = ixp.declarerank2("uu_dDD","sym12")

# Step 3: Define the C parameter wavespeed. The `wavespeed`
#         variable is a proper SymPy variable, so it can be
#         used in below expressions. In the C code, it acts
#         just like a usual parameter, whose value is 
#         specified in the parameter file.
wavespeed = par.Cparameters("REAL",thismodule,"wavespeed")

# Step 4: Define right-hand sides for the evolution.
uu_rhs = vv
vv_rhs = 0
for i in range(DIM):
    vv_rhs += wavespeed*wavespeed*uu_dDD[i][i]

vv_rhs = sp.simplify(vv_rhs)
    
# Step 5: Generate C code for scalarwave evolution equations,
#         print output to the screen (standard out, or stdout).
fin.FD_outputC("stdout",
               [lhrh(lhs=gri.gfaccess("out_gfs","UU_rhs"),rhs=uu_rhs),
                lhrh(lhs=gri.gfaccess("out_gfs","VV_rhs"),rhs=vv_rhs)])


/*
 *  Original SymPy expressions:
 *  "[const double uu_dDD00 = invdx0**2*(-5*uu/2 + 4*uu_i0m1/3 - uu_i0m2/12 + 4*uu_i0p1/3 - uu_i0p2/12),
 *    out_gfs[IDX2(UU_rhs, i0)] = vv,
 *    out_gfs[IDX2(VV_rhs, i0)] = uu_dDD00*wavespeed**2]"
 */
{
   const double uu_i0m2 = in_gfs[IDX2(UUGF, i0-2)];
   const double uu_i0m1 = in_gfs[IDX2(UUGF, i0-1)];
   const double uu = in_gfs[IDX2(UUGF, i0)];
   const double uu_i0p1 = in_gfs[IDX2(UUGF, i0+1)];
   const double uu_i0p2 = in_gfs[IDX2(UUGF, i0+2)];
   const double vv = in_gfs[IDX2(VVGF, i0)];
   const double uu_dDD00 = pow(invdx0, 2)*(-(5.0 / 2.0)*uu + ((4.0 / 3.0))*uu_i0m1 - (1.0 / 12.0)*uu_i0m2 + ((4.0 / 3.0))*uu_i0p1 - (1.0 / 12.0)*uu_i0p2);;
   out_gfs[IDX2(UU_rhs, i0)] = vv;;
   out_gfs[IDX2(VV_rhs, i0)] = uu_dDD00*pow(wavespeed, 2);;
}

